In [ ]:
# IIT2019189

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [14]:
dataset_url = "https://raw.githubusercontent.com/Nidhikamewar15/ML/main/Housing%20Price%20data%20set.csv"
dataset = pd.read_csv(dataset_url)
dataset.head() 

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,1,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,2,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,3,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,4,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,5,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [ ]:
Area_of_floor = dataset['lotsize']
Number_Of_Bedrooms = dataset['bedrooms']
Number_Of_Bathrooms = dataset['bathrms']
Price_of_House = dataset['price']
Y_Matrix = dataset['price']
Y_Matrix = np.array(Y_Matrix)
Y_Matrix = Y_Matrix.reshape(Y_Matrix.shape[0], 1)

##feature scaling

In [ ]:
# Now here we are performing feature scanning on Area_of_floor

Mean_of_floor_area = np.mean(Area_of_floor)
Max_of_floor_area = max(Area_of_floor)
Min_of_floor_area = min(Area_of_floor)
Scaled_FloorArea = []
for i in Area_of_floor:
	Scaled_FloorArea.append((i - Mean_of_floor_area) / (Max_of_floor_area - Min_of_floor_area))


##Linear Weighted Regression 

In [ ]:
def calculate_error(Y_Matrix, Y_pred_set):
    err = 0
    for i in range(len(Y_Matrix)):
        err += abs(Y_Matrix[i] - Y_pred_set[i]) / Y_Matrix[i]
    err = err / len(Y_Matrix)
    return err * 100

def kernel(X_Matrix, xi, HyperParameterTau):
    return np.exp(-np.sum((xi - X_Matrix) ** 2, axis = 1) / (2 * HyperParameterTau * HyperParameterTau))

def LocallyWeightedLR(X_Matrix, xi, Y_Matrix, HyperParameterTau):
	TransposeMatrixX = np.transpose(X_Matrix)
	W = kernel(X_Matrix, xi, HyperParameterTau)
	MatrixXTransposeW = TransposeMatrixX * W
	MatrixXTransposeWX = np.matmul(MatrixXTransposeW, X_Matrix)
	InverseMatrixXTransposeWX = np.linalg.pinv(MatrixXTransposeWX)
	InverseMatrixXTransposeWXMatrixXTransposeW = np.matmul(InverseMatrixXTransposeWX, MatrixXTransposeW)
	InverseMatrixXTransposeWXMatrixXTransposeWY = np.matmul(InverseMatrixXTransposeWXMatrixXTransposeW, Y_Matrix)
	InverseMatrixXTransposeWXMatrixXTransposeWYTranspose = np.transpose(InverseMatrixXTransposeWXMatrixXTransposeWY)
	return InverseMatrixXTransposeWXMatrixXTransposeWYTranspose.dot(xi)


In [ ]:
X_Matrix = []
for i in range(len(Area_of_floor)):
	X_Matrix.append([1, Scaled_FloorArea[i], Number_Of_Bedrooms[i], Number_Of_Bathrooms[i]])
X_Matrix = np.array(X_Matrix)

Taus = [0.1, 0.01, 0.02, 0.001, 0.0001, 0.00001]
for Tau in Taus:
  print("Locally Weighted Linear Regression for Tau = " + str(Tau))
  pred = []
  for i in range(X_Matrix.shape[0]):
	  y_pred_test = LocallyWeightedLR(X_Matrix, X_Matrix[i], Y_Matrix, Tau)
	  pred.append(y_pred_test)
  print("Mean absolute % error is : " + str(calculate_error(Y_Matrix,pred)))


Locally Weighted Linear Regression for Tau = 0.1
Mean absolute % error is : [20.14065581]
Locally Weighted Linear Regression for Tau = 0.01
Mean absolute % error is : [16.41583941]
Locally Weighted Linear Regression for Tau = 0.02
Mean absolute % error is : [18.12596846]
Locally Weighted Linear Regression for Tau = 0.001
Mean absolute % error is : [7.56066413]
Locally Weighted Linear Regression for Tau = 0.0001
Mean absolute % error is : [5.40856556]
Locally Weighted Linear Regression for Tau = 1e-05
Mean absolute % error is : [5.40730554]


##Batch gradient descent with regularization 

In [ ]:
FloorAreaTrain = Scaled_FloorArea[:383]
NoOfBathroomsTrain = Number_Of_Bathrooms[:383]
NoOfBedroomsTrain = Number_Of_Bedrooms[:383]
PriceTrain = Price_of_House[:383]
FloorAreaTest = Area_of_floor[383:]
NoOfBathroomsTest = Number_Of_Bathrooms[383:]
NoOfBedroomsTest = Number_Of_Bedrooms[383:]
PriceTest = Price_of_House[383:]


In [ ]:
X_train_set = []
for i in range(383):
	X_train_set.append([1, Scaled_FloorArea[i], Number_Of_Bedrooms[i], Number_Of_Bathrooms[i]])
 
X_test_set = []
n = len(FloorAreaTest)
for i in range(383,383+n):
  X_test_set.append([1,Scaled_FloorArea[i],Number_Of_Bedrooms[i],Number_Of_Bathrooms[i]])

In [ ]:
#learning rate is abbreviated as l_r
l_r = 0.001  

#lambda value is abbreviated as lamda
lamda = -120 
x = np.array(X_train_set)
y = np.array(PriceTrain)
w = np.array(([0],[0],[0],[0]))
y = y.reshape(len(y),1)

def prod_function(w,x):
    res = 0
    for i in range(w.shape[0]):
        res = res + w[i][0]*x[i]
    return res

def grad_function(x,w,y):
    res = np.zeros(w.shape[0])
    for i in range(x.shape[0]):
        res = (res+(prod_function(w,x[i])-y[i][0])*x[i])
    res = res.reshape(w.shape[0],1)
    return res

m = x.shape[0]
for i in range(5000):
  flag = 1-((l_r*lamda)/m)
  w = flag * w - l_r*grad_function(x,w,y)/m

print("Coff's are : ",w)


# Steps to find mean absolute % error
x_test_set = np.array(X_test_set)
y_test_set = np.array(PriceTest)
y_pred_set = np.dot(x_test_set,w)

n = x_test_set.shape[0]
m = 0

for i in range(n):
  m = m + abs(y_pred_set[i][0] - y_test_set[i])/y_test_set[i]
m = m * 100/n
print("Mean absolute % error is ",m)


Coff's are :  [[11027.93238871]
 [19272.80715199]
 [ 5262.5362372 ]
 [31018.09747139]]
Mean absolute % error is  18.84547313581711


##stochastic gradient descent with regulariation

In [ ]:
X_train_set = []
for i in range(383):
	X_train_set.append([1, Scaled_FloorArea[i], Number_Of_Bedrooms[i], Number_Of_Bathrooms[i]])
 
X_test_set = []
n = len(FloorAreaTest)
for i in range(383,383+n):
  X_test_set.append([1,Scaled_FloorArea[i],Number_Of_Bedrooms[i],Number_Of_Bathrooms[i]])

In [ ]:
# learning rate is abbreviated as l_r
l_r = 0.0005

#lambda as lamda
lamda = -10

x = np.array(X_train_set)
y = np.array(PriceTrain)
w = np.array([0,0,0,0])
y = y.reshape(len(y),1)

print("Initial coeff's are as follows : ",w)

def prod_function(w,x):
    res = 0
    for i in range(x.shape[0]):
        res = res + w[i]*x[i]
    return res


for itr in range(20):
  for i in range(x.shape[0]):
    grad = (prod_function(w,x[i]) - y[i][0])*x[i]
    flag = 1-((lamda * l_r) / x.shape[0])
    w = flag*w - l_r * grad

print("Coeff's are as follows :",w)


# Mean absolute % error
w = w.reshape((4,1))
x_test_set = np.array(X_test_set)
y_test_set = np.array(PriceTest)
y_pred_set = np.dot(x_test_set,w)

n = x_test_set.shape[0]
m = 0

for i in range(n):
  m = m + abs(y_pred_set[i][0] - y_test_set[i])/y_test_set[i]
m = m * 100/n
print("Mean absolute % error",m)

Initial coeff's are as follows :  [0 0 0 0]
Coeff's are as follows : [ 8099.53322869  6266.87283801 13353.66990034 17698.12315756]
Mean absolute % error 18.655780791680915


##Mini Batch Gradient Descent with Regulatization 

In [ ]:
X_train_set = []
for i in range(383):
	X_train_set.append([1, Scaled_FloorArea[i], Number_Of_Bedrooms[i], Number_Of_Bathrooms[i]])
 
X_test_set = []
n = len(FloorAreaTest)
for i in range(383,383+n):
  X_test_set.append([1,Scaled_FloorArea[i],Number_Of_Bedrooms[i],Number_Of_Bathrooms[i]])

In [ ]:
b_s = 20 
l_r = 0.002
lamda = -20

x = np.array(X_train_set)
y = np.array(PriceTrain)
w = np.array([0,0,0,0])
y = y.reshape(len(y),1)

def prod_function(w,x):
    res = 0
    for i in range(x.shape[0]):
        res = res + w[i]*x[i]
    return res

def grad_function(x,w,y,st,b_s):
    res = np.zeros(w.shape[0])
    for i in range(st,st+b_s):
        if(i>=x.shape[0]):
          break
        res=(res+(prod_function(w,x[i])-y[i][0])*x[i])
    return res

for itr in range(50):
  for i in range(b_s):
    flag = 1-((lamda*l_r)/b_s)
    w = flag * w - (l_r*grad_function(x,w,y,b_s*i,b_s))/b_s
print("Coeff's are as follows : ",w)



# Mean absolute % error
w = w.reshape((4,1))
x_test_set = np.array(X_test_set)
y_test_set = np.array(PriceTest)
y_pred_set = np.dot(x_test_set,w)

n = x_test_set.shape[0]
m = 0

for i in range(n):
  m = m + abs(y_pred_set[i][0] - y_test_set[i]) / y_test_set[i]
m = m * 100/n
print("Mean absolute % error : " , m)

Coeff's are as follows :  [ 9280.88414526 10001.13873579 10055.23366084 25309.01693085]
Mean absolute % error :  17.971898815464968
